# Web scraping con Selenium

Hay ocasiones en las que la descarga de la página/sitio web y su posterior procesamiento con BeautifulSoup no es suficiente. Esto puede deberse a que el sitio del que queremos extraer los datos es una SPA (_Single Page Application_), está protegido por un login (mucho cuidado con los Términos de Uso) o a que necesitamos, por ejemplo, interactuar con el sitio, ya sea rellenando formularios o pulsando en algún enlace.

A modo de ejemplo vamos a buscar cuántas recetas de comida española hay en el sitio web Yummly. Se puede acceder a estas recetas a través de esta URL:

<http://www.yummly.com/recipes?q=&allowedCuisine=cuisine^cuisine-spanish&noUserSettings=true>

In [1]:
from bs4 import BeautifulSoup
import requests

url = "http://www.yummly.com/recipes?q=&allowedCuisine=cuisine^cuisine-spanish&noUserSettings=true"

# Realizamos la petición a la web
req = requests.get(url)

# Comprobamos que la petición nos devuelve un Status Code = 200
statusCode = req.status_code
if statusCode == 200:

    # Pasamos el contenido HTML de la web a un objeto BeautifulSoup()
    html = BeautifulSoup(req.text, "lxml")
    recipes = html.select(".recipe-card")
    print ("Número de recetas: {}".format(len(recipes)))
    
else:
    print (statusCode)


Número de recetas: 36


Como podemos ver, solo se han recuperado 36 recetas. Si accedemos a la página entonces veremos que al hacer scroll el número de recetas aumenta. 

Para este tipo de problemas es necesario utilizar una alternativa a Beautiful Soup que nos permita interactuar con la página web. Aunque hay muchas y muy variadas alternativas aquí vamos a utilizar [Selenium](http://www.seleniumhq.org/docs/). Esta librería sirve para la automatización de tareas que hacen uso de un navegador. En particular, se suele utilizar para hacer pruebas sobre interfaces web, aunque nosotros vamos a utilizarlo interactuar y extraer información de algunos sitios web. Recomendamos revisar [la documentación de Selenium para Python](http://selenium-python.readthedocs.io/) para sacar el máximo partido a esta librería.

## Instalación

Para poder utilizar Selenium es necesario:

1. Instalar la librería Selenium. Podemos instalarla desde el propio notebook con los gestores de paquetes pip (python) o conda (Anaconda).
2. Descargar el/los drivers para controlar un nevagador. Durante este taller usaremos el driver de Chrome, aunque podremos utilizar otros:

    + Chrome: <https://sites.google.com/a/chromium.org/chromedriver/downloads>
    + Edge: <https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/>
    + Firefox: <https://github.com/mozilla/geckodriver/releases>
    + Safari: <https://webkit.org/blog/6900/webdriver-support-in-safari-10/>



In [3]:
# Descomenta y ejecuta una de las dos alternativas (o ejecútalos desde una consola de comandos):
# 1. Instalar desde Anaconda
#!conda install -c conda-forge selenium
# 2. Instalar con pip
#!pip install -U selenium

## Ejecución básica

La filosofía de Selenium es diferente ya que, en este caso, lo que vamos a hacer es programar la navegación automática por un sitio web. Para ello, Selenium lanza un navegador que controlaremos desde Python.

Para lanzar el navegador:


In [4]:
from selenium import webdriver

url = "http://www.yummly.com/recipes?q=&allowedCuisine=cuisine^cuisine-spanish&noUserSettings=true"
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
# Si usamos Firefox:
# driver = webdriver.Firefox(executable_path='../seleniumDrivers/geckodriver')
driver.get(url)

Una vez que tenemos el navegador abierto y la página cargada, podemos navegar por la página mediante el repertorio de instrucciones `find_*` [que proporciona Selenium](http://selenium-python.readthedocs.io/locating-elements.html):

In [5]:
recipes = driver.find_elements_by_class_name("recipe-card")
print ("Número de recetas: {}".format(len(recipes)))

Número de recetas: 36


Terminaremos cerrando el navegador:

In [7]:
driver.quit()

Antes de abordar el ejemplo de Yummly vamos a realizar iteraciones más sencillas con otras páginas:

## Ejemplo: Generación de contenidos mediante la interacción con una web

Selenium nos facilita la interacción con la web. Vamos a poner como ejemplo la interacción con una herramienta de generación de [contraseñas mediante htdigest](https://httpd.apache.org/docs/2.4/programs/htdigest.html) (<https://websistent.com/tools/htdigest-generator-tool/>). Vamos primeramente a rellenar el formulario:

In [24]:
from selenium import webdriver

url = "https://websistent.com/tools/htdigest-generator-tool/"
usuario = "miUsuario"

driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get(url)

element = driver.find_element_by_id("uname")
element.send_keys(usuario)

Si vamos al navegador veremos que hemos rellenado el primer cambio del formulario. A continuación rellenamos el resto.

In [25]:
element = driver.find_element_by_id("realm")
element.send_keys("miRealm")

element = driver.find_element_by_id("word1")
element.send_keys("12345")

element = driver.find_element_by_id("word2")
element.send_keys("12345")

Finalmente, buscamos el botón y lo pulsamos:

In [26]:
driver.find_element_by_id("generate").click();

Vemos que al final de la página se ha generado un texto con el resultado de la ejecución. Vamos a quedarnos con él:

In [27]:
output = driver.find_element_by_id("output").text
print (output[output.find(usuario):])
driver.quit()

miUsuario:miRealm:f183ef39e2332c681a3702eee9f8a9ac


Probemos a ejecutarlo todo de una sola vez:

In [30]:
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get(url)

element = driver.find_element_by_id("uname")
element.send_keys(usuario)

element = driver.find_element_by_id("realm")
element.send_keys("miRealm")

element = driver.find_element_by_id("word1")
element.send_keys("12345")

element = driver.find_element_by_id("word2")
element.send_keys("12345")

driver.find_element_by_id("generate").click();

output = driver.find_element_by_id("output").text
print (output[output.find(usuario):])
driver.quit()

.


Vemos que la ejecución no ha funcionado correctamente. Esto se debe a que hay tareas (como la generación del resultado) que tardan un tiempo en ejecutarse, por lo que tenemos que esperar a que terminen para poder extraer un resultado. Para ello hay que implementar esperas ([Waits, en Selenium](http://selenium-python.readthedocs.io/waits.html)) en nuestro código.

La primera y más sencilla es hacer una pausa antes de ejecutar una instrucción:

In [33]:
import time
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get(url)

element = driver.find_element_by_id("uname")
element.send_keys(usuario)

element = driver.find_element_by_id("realm")
element.send_keys("miRealm")

element = driver.find_element_by_id("word1")
element.send_keys("12345")

element = driver.find_element_by_id("word2")
element.send_keys("12345")

driver.find_element_by_id("generate").click();

# Esperamos 2 segundos antes de buscar el elemento
time.sleep(2)

output = driver.find_element_by_id("output").text
print (output[output.find(usuario):])
driver.quit()

miUsuario:miRealm:f183ef39e2332c681a3702eee9f8a9ac


Otra opción es utilizar las esperas explícitas de Selenium, con las que pedimos al driver que ejecute una método de manera constante durante durante un periodo de máximo de tiempo hasta que nos devuelva un determinado valor. En caso de que se sobrepase el tiempo máximo de espera entonces lanza una excepción. En nuestro ejemplo, vemos que el resultado siempre aparece en el mismo elemento y que cuando pulsamos en el botón primero aparece el texto "Loading". Por tanto, vamos a esperar hasta que desaparezca ese texto para extraer el resultado correcto:

In [41]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get(url)

element = driver.find_element_by_id("uname")
element.send_keys(usuario)

element = driver.find_element_by_id("realm")
element.send_keys("miRealm")

element = driver.find_element_by_id("word1")
element.send_keys("12345")

element = driver.find_element_by_id("word2")
element.send_keys("12345")

driver.find_element_by_id("generate").click();

try:
    # Esperamos como máximo 10 segundos mientras esperamos a que desaparezca el texto "Loading"
    WebDriverWait(driver, 10).until_not(lambda driver: driver.find_element_by_id("output").text.startswith("Loading"))

    output = driver.find_element_by_id("output").text
    print (output[output.find(usuario):])

except TimeoutException:
    print("No se ha podido generar el realm o la página ha tardado demasiado")
    
finally:
    driver.quit()

miUsuario:miRealm:f183ef39e2332c681a3702eee9f8a9ac


## Ejemplo: Yummly

Vamos a atacar nuestro ejemplo motivador usando Selenium. Lo primero que tenemos que hacer es analizar detenidamente el DOM de la web para saber cómo hacer el scroll. En este caso no es sencillo, ya que no hay que hacer scroll sobre la ventana del navegador sino sobre un elemento del mismo. Además, no podemos hacer click en los botones de scroll. Sin embargo, otra cosa que nos permite Selenium es ejecutar scripts en el navegador. Esto es lo que vamos a utilizar para la extracción de los resultados:

In [60]:
from selenium import webdriver
import time
url = "http://www.yummly.com/recipes?q=&allowedCuisine=cuisine^cuisine-spanish&noUserSettings=true"
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get(url)

time.sleep(5)

recipes = driver.find_elements_by_class_name("recipe-card")
print ("Número de recetas: {}".format(len(recipes)))

# Realizamos scroll ejecutando código javascript en el navegador
driver.execute_script('cookbook = document.getElementsByClassName("cookbook")[0];')
driver.execute_script('maxScroll = document.getElementsByClassName("RecipeGrid")[0].clientHeight;')
driver.execute_script('cookbook.scrollTo(0, maxScroll);')

time.sleep(5)

recipes = driver.find_elements_by_class_name("recipe-card")
print ("Número de recetas: {}".format(len(recipes)))

driver.quit()

Número de recetas: 36
Número de recetas: 66


La automatización de este scraping no es nada sencilla ya que la página usa AJAX de forma muy profusa, con grandes retardos, lo que nos perjudica a la hora de extraer la información. Una posible solución puede ser la siguiente:

In [46]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
import time

def heightHasChanged(driver, lastHeight):
    """
    Comprueba si, tras hacer scroll, el tamaño del panel en el que están las recetas ha cambiado
    """
    new_height = driver.execute_script('return document.getElementsByClassName("RecipeGrid")[0].clientHeight;')
    return last_height!= new_height

url = "http://www.yummly.com/recipes?q=&allowedCuisine=cuisine^cuisine-spanish&noUserSettings=true"
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get(url)

last_height = 0

time.sleep(5)

try:
    while True:
        # Realizamos scroll ejecutando código javascript en el navegador
        driver.execute_script('cookbook = document.getElementsByClassName("cookbook")[0];')
        driver.execute_script('maxScroll = document.getElementsByClassName("RecipeGrid")[0].clientHeight;')
        driver.execute_script('cookbook.scrollTo(0, maxScroll);')

        # Esperamos hasta que se actualice el scroll o lleguemos hasta el final
        WebDriverWait(driver, 10).until(lambda driver: heightHasChanged(driver,last_height))
        
        last_height = driver.execute_script('return document.getElementsByClassName("RecipeGrid")[0].clientHeight;')

except TimeoutException:
    # Se supone que hemo hecho scroll hasta el final 
    recipes = driver.find_elements_by_class_name("recipe-card")
    print ("Número de recetas: {}".format(len(recipes)))

finally:
    driver.quit()

Número de recetas: 2221


Finalmente, podemos combinar el uso de Selenium con BeautifulSoup. Para ello hemos de tener en cuenta que cada uno de los elementos extraídos con `find_*` disponen de los atributos `innerHTML` y `outerHTML`, que contienen el texto HTML del elemento. Podemos procesar este HTML con Beautiful Soup, si nos resulta más cómodo:

In [61]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup

url = "http://www.yummly.com/recipes?q=&allowedCuisine=cuisine^cuisine-spanish&noUserSettings=true"
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get(url)

time.sleep(5)

# Realizamos scroll ejecutando código javascript en el navegador
driver.execute_script('cookbook = document.getElementsByClassName("cookbook")[0];')
driver.execute_script('maxScroll = document.getElementsByClassName("RecipeGrid")[0].clientHeight;')
driver.execute_script('cookbook.scrollTo(0, maxScroll);')

time.sleep(5)

recipeContainer = driver.find_element_by_class_name("RecipeContainer")
html = BeautifulSoup(recipeContainer.get_attribute('outerHTML'), 'lxml')
driver.quit()

# A partir de aquí, lo tratamos con BS4
recipes = html.select(".recipe-card h2.card-title a")
print("Número de recetas: {}".format(len(recipes)))
for title in recipes:
    print(title.text)



Número de recetas: 50
Portuguese Fish Stew
...
Gazpacho


## Extra: Acceso a páginas con autenticación (Campus Virtual)


Como al ejecutar Selenium estamos lanzando un navegador, esto significa que tenemos toda la información relacionada con la sesión que estamos ejecutando, cookies, etc. Vamos a hacer una última prueba accediendo al Campus Virtual de la UCM con nuestro usuario y contraseña.

In [62]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import getpass

# Cargamos el driver y el Campus Virtual
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')
driver.get('http://www.ucm.es/campusvirtual')
assert 'Campus Virtual - UCM' in driver.title
driver.find_element_by_name('submit').click()

# Usuario y contraseña
user = '' # Indica aquí tu usuario (la contraseña nos la pedirá por la consola)
password = getpass.getpass('Introduzca contraseña (%s):' % user)
driver.find_element_by_id('username').send_keys(user)
driver.find_element_by_id('password').send_keys(password)
driver.find_element_by_class_name('btn').click()

# Esperamos hasta que lleguemos a la página de Mi Campus (hay varias redirecciones)
WebDriverWait(driver, 10).until(lambda driver: 'Mi Campus' in driver.title)

Introduzca contraseña ():


True

Una vez dentro, vamos a listar todos los cursos en los que estamos dados de alta en el Campus

In [64]:
cursos = driver.find_elements_by_css_selector("#panel-sliders table.adminlist th.left a")
print("Número de cursos: {}".format(len(cursos)))
for curso in cursos:
    print("Curso: {}({})".format(curso.text,curso.get_attribute("href")))

Número de cursos: 125
Curso:  ANÁLISIS DE REDES SOCIALES(https://cv4.ucm.es/moodle/auth/saml/index.php?urlCurso=https://cv4.ucm.es/moodle/course/view.php?id=90511)
...
Curso: (https://cv4.ucm.es/moodle/auth/saml/index.php?urlCurso=https://cv4.ucm.es/moodle/course/view.php?id=9783)


Un detalle curioso es que solo vemos los nombres de los cursos **que están visibles en el navegador**. Esto se debe a que muchos de los cursos están ocultos mediante lo que se conoce como un "acordeón". Esto nos permite hacer lo que se conoce como **screen scraping**, es decir, solo podemos cosechar los datos que puede ver un usuario humano interactuando con el navegador.

Por este motivo, en este caso nos puede resultar más interesante procesar este catálogo con BS4 (que procesa el HTML tal cual, ignorando lo que ve o no ve el usuario).

In [67]:
from bs4 import BeautifulSoup

html = BeautifulSoup(driver.find_element_by_id("panel-sliders").get_attribute('outerHTML'), 'lxml')
driver.quit()
cursos = html.select("table.adminlist th.left a")
print("Número de cursos: {}".format(len(cursos)))
for curso in cursos:
    print("Curso: {}({})".format(curso.text,curso["href"]))

Número de cursos: 125
Curso:  ANÁLISIS DE REDES SOCIALES(https://cv4.ucm.es/moodle/auth/saml/index.php?urlCurso=https://cv4.ucm.es/moodle/course/view.php?id=90511)
...
Curso: Espacio Coordinación de la Facultad de Informática(https://cv4.ucm.es/moodle/auth/saml/index.php?urlCurso=https://cv4.ucm.es/moodle/course/view.php?id=9783)
